# Crawl 天下雜誌 news from several categories

In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [31]:
# 11 news categories
#news_links =['aipl', 'ait', 'aspt', 'asc', 'aie', 'amov','ahel','aopl','asoc','acul','acn']
#news_categories=['政治','科技','運動','證卷','產經','娛樂','生活','國際','社會','文化','兩岸']
news_links =['ahel','aopl','asoc','acul','acn']
news_categories=['生活','國際','社會','文化','兩岸']
base_url = 'https://www.cna.com.tw/list/'

In [32]:
import requests
import re
from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import datetime, timedelta
from fake_useragent import UserAgent

In [33]:
# 設定 User-Agent
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

In [34]:
user_agent = UserAgent()

In [37]:
%%time
# 存放資料之變數
links = []
titles = []
dates = []
contents = []
categories = []
item_id = []
photo_links = []


# 這裡我們需要用for迴圈爬取11個類別
# Here we need to crawl 11 categories using for loop
for i, url_short_name in enumerate(news_links):  #針對每一類 共有11類

    category = news_categories[i] #'政治' #類別名稱紀錄起來 

    # Categorical url link
    category_url = base_url + url_short_name + '.aspx'
    print("Getting categorical news:", category)

    # Request the categorical news page
    # req = requests.get(category_url)
    req = requests.get(category_url, headers={ 'user-agent': user_agent.random }, timeout=5)
    page = BeautifulSoup(req.text, 'lxml')
    #print(page)

    # 抓新聞列表
    items = page.find('ul', {'id': re.compile("jsMainList\w*")}).findAll('li')
    #print(items)


    # Let's start to crawl the news in the first page for that category
    serial_no = 1
    for item_j, item in enumerate(items,start=1): #針對每一篇項目 抓其細節
        title = item.find('h2').text
        print(serial_no,'--', title )
        
        link = item.find('a').get('href')
        print(link)

        try:
            if item.find('img').has_attr('data-src'):
                photo_link = item.find('img').get('data-src')
            else:
                photo_link = item.find('img').get('src')
        except:
            photo_link = ''
        
        # get date and convert it with the format of  "2022-04-20"
        news_date = item.find('div', {'class':"date"}).text
        # print(news_date)
        dtime = datetime.strptime(news_date, '%Y/%m/%d %H:%M')
        # conver to the format of  "2022-04-20"
        news_date_str = dtime.strftime("%Y-%m-%d") # keep only date, remove the time.

        categories.append(category)
        titles.append(title)
        links.append(link)
        photo_links.append(photo_link)
        dates.append(news_date_str)
 
        # item_id.append("cna_" + tstr + "_" + str(serial_no))
        # conver to the format of  "20220420"
        dtstr = dtime.strftime("%Y%m%d") # keep only date, remove the time.
        # format of serial number: aipl_20220314_1
        
        item_id.append(url_short_name + "_" + dtstr + "_" + str(serial_no))
        serial_no += 1

        #抓新聞內容
        # 最新的新聞列表之新聞連結:開頭缺少"https://www.cna.com.tw"
        if link.startswith("/news"):
            link = "https://www.cna.com.tw"+link
        
        print(link)
        req = requests.get(link, headers={ 'user-agent': user_agent.random }, timeout=5)
        
        page = BeautifulSoup(req.text,'lxml')

        # Find content
        content = page.find('div',{'class':"paragraph"})

        # Remove unnecessary content
        # There are some words we don't like to analyze, They should be removed.
        # Remove "透過 Google News追蹤中央社"
        try:
            content.find('div', {'class':"gmailNews"}).decompose()
        except:
            pass
        # 額外插入的延伸閱讀區塊-->移除之
        try:
            content.find('div', {'class':"paragraph moreArticle"}).decompose()
        except:
            pass
        # 額外插入的訂閱連結-->移除之
        try:
            content.find('div', class_="SubscriptionInner mySubscriptionInner").decompose()
        except:
            pass

        # 額外插入的圖片與文字說明區塊-->移除之
        try:
            for media in content.findAll('div', {'class':"media"}):
                media.decompose()
        except:
            pass

        # Remove unnecessary content
        cont = content.text
        cont = re.sub('（中央社.+?）','', cont)
        cont = re.sub('（編輯：.*','', cont) 
        cont = re.sub('（譯者：.*','', cont) 
        
        contents.append(cont)

        if item_j >= 4: # Here we crawl only 4 pieces of news for each category, in order to save time.
            break # 跳開一層迴圈
        
        time.sleep(10)  # 遵守爬蟲禮節，請小睡一下


Getting categorical news: 生活
1 -- 健保砸逾42億解急診壅塞 護理加薪4提案通過拚5月上路
/news/ahel/202503120382.aspx
https://www.cna.com.tw/news/ahel/202503120382.aspx
2 -- 今彩539第114064期 頭獎槓龜
/news/ahel/202503120395.aspx
https://www.cna.com.tw/news/ahel/202503120395.aspx
3 -- 今彩539第114064期開獎
/news/ahel/202503120374.aspx
https://www.cna.com.tw/news/ahel/202503120374.aspx
4 -- 邱泰源與醫療工會開會 達擬跨科支援指引等共識
/news/ahel/202503120365.aspx
https://www.cna.com.tw/news/ahel/202503120365.aspx
Getting categorical news: 國際
1 -- 英國鋼鋁遭美徵收25%關稅 英相施凱爾透露不排除反制
/news/aopl/202503120410.aspx
https://www.cna.com.tw/news/aopl/202503120410.aspx
2 -- 加拿大報復川普 3/13起對價值逾6800億美國商品加關稅
/news/aopl/202503120407.aspx
https://www.cna.com.tw/news/aopl/202503120407.aspx
3 -- 美國向全球徵收鋼鋁25%關稅 英國不放棄爭取豁免
/news/aopl/202503120391.aspx
https://www.cna.com.tw/news/aopl/202503120391.aspx
4 -- 揭中國海纜佈局東非智慧城市 台藝術家致穎柏林展覽
/news/aopl/202503120379.aspx
https://www.cna.com.tw/news/aopl/202503120379.aspx
Getting categorical news: 社會
1 -- 屏東酒後口角衝突 女持酒瓶攻擊友人致死
/news/asoc/2025

In [38]:
titles

['健保砸逾42億解急診壅塞 護理加薪4提案通過拚5月上路',
 '今彩539第114064期 頭獎槓龜',
 '今彩539第114064期開獎',
 '邱泰源與醫療工會開會 達擬跨科支援指引等共識',
 '英國鋼鋁遭美徵收25%關稅 英相施凱爾透露不排除反制',
 '加拿大報復川普 3/13起對價值逾6800億美國商品加關稅',
 '美國向全球徵收鋼鋁25%關稅 英國不放棄爭取豁免',
 '揭中國海纜佈局東非智慧城市 台藝術家致穎柏林展覽',
 '屏東酒後口角衝突 女持酒瓶攻擊友人致死',
 '國道1號汐五高架堤頂段火燒車 幸未釀傷亡',
 '沈慶京稱看守所室友幫換引流管 北所駁：皆專業處置',
 '澎湖花嶼村長補選當選人涉賄  澎檢偵結起訴',
 '台漫畫家左萱「芭蕉的芽」2  獲日本國際漫畫賞銀獎',
 '國家攝影中心推影像盛宴  攝影新銳作品上桌',
 '國父逝世100週年特展 展出當年醫書院高分成績單',
 '故宮北院二館啟用展 美國大都會博物館典藏6月登台',
 '國台辦挺藝人轉央視貼文 陸委會：中共沒資格談言論自由',
 '打破出事謠言 趙樂際現身看望新聞工作者',
 '阿里巴巴主席：民營企業座談會釋放積極訊號',
 '香港3所中學師生到大陸交流後出現腸胃炎']

In [39]:
contents


['為解決急診壅塞，健保署提出增加急診及重症支付標準、給急診護理人員加薪、強化離島急救責任醫院等4招解方，醫院總額研商會今天通過，最快5月上路，年挹注逾新台幣42億元。急診近來面臨前所未有壅塞，衛生福利部中央健康保險署今天在醫療給付費用醫院總額研商議事會討論解方，包含急診支付標準與留觀床支付、急救責任醫院加護病房給付、強化離島急救責任醫院給付與急性一般病床護理費調升等4案。健保署長石崇良今天晚間接受媒體聯訪表示，與解決急診壅塞有關的4項提案全數通過，合計挹注42.35億元，將儘速送入共擬會與健保會，拚5月起開始執行。這次不是只挹注，還有附帶任務指標，共同指標是醫院必須落實醫護人員加薪，其他還有急診壅塞指標改善、加護病房開床率等。針對健保署7日開會與醫界未達成共識的急性一般病床護理費調整，石崇良表示，確定醫學中心第1天住院急性一般病床護理費調幅12.4%，1027點調至1154點，區域醫院調幅14%，905點調至1032點，地區醫院調14.9%，從850點調至977點。石崇良說，醫學中心第2天住院急性一般病床護理費調幅12.3%，從790點調至888點，區域醫院調幅14.1%，696點調至794點，地區醫院調15%，從654點調至752點，挹注24.97億元，盼改善護理薪資環境，留住護理人力，避免關床，以利解決急診患者排隊等住院。這次還有通過強化離島急救責任醫院，石崇良表示，未來將新增離島一般急救責任醫院給付調升標準，所有急診案件皆按照表定點數加給10%，離島中度急救責任醫院所有急診案件給付增加30%、離島重度急救責任醫院調升50%，鼓勵離島醫院提升緊急醫療能力。石崇良說，急診醫師診察費和護理費確定拆分，按照檢傷級數差異，各調增10%，以最嚴重的檢傷分類1級為例，診察費由1223點調到1345點，護理費從577點調到635點；觀察床診察費調至468點，護理費調升6成，留觀第1天自571點調升至914點、第2天從175點調至703點，增預算11億元。急救責任醫院加護病房給付，石崇良說，若是全重度級的急救責任醫院，加護病房給付比照醫學中心；中度級急救責任醫院則比照區域醫院；盼52家重度級急救責任醫院中26家區域醫院，能協助分擔重症病人收治，健保預估新增約5.87億元支出。除了解決急診壅塞，這次研商議事會考量因應台灣少子化與產婦高齡化，高風險生產個案將轉至醫院，通過醫院生產

In [40]:
item_id


['ahel_20250312_1',
 'ahel_20250312_2',
 'ahel_20250312_3',
 'ahel_20250312_4',
 'aopl_20250312_1',
 'aopl_20250312_2',
 'aopl_20250312_3',
 'aopl_20250312_4',
 'asoc_20250312_1',
 'asoc_20250312_2',
 'asoc_20250312_3',
 'asoc_20250312_4',
 'acul_20250312_1',
 'acul_20250312_2',
 'acul_20250312_3',
 'acul_20250312_4',
 'acn_20250312_1',
 'acn_20250312_2',
 'acn_20250312_3',
 'acn_20250312_4']

In [41]:
data = zip(item_id, dates, categories, titles, contents, links, photo_links)
df = pd.DataFrame(list(data), columns=['item_id','date','category','title','content','link','photo_link'])

In [42]:
df.head(2)

item_id        date category                        title  \
0  ahel_20250312_1  2025-03-12       生活  健保砸逾42億解急診壅塞 護理加薪4提案通過拚5月上路   
1  ahel_20250312_2  2025-03-12       生活           今彩539第114064期 頭獎槓龜   

                                             content  \
0  為解決急診壅塞，健保署提出增加急診及重症支付標準、給急診護理人員加薪、強化離島急救責任醫院等...   
1  今彩539第114064期開獎，中獎號碼06、14、27、30、33；派彩結果，頭獎槓龜。貳...   

                           link  \
0  /news/ahel/202503120382.aspx   
1  /news/ahel/202503120395.aspx   

                                          photo_link  
0  https://imgcdn.cna.com.tw/www/webphotos/WebCov...  
1

In [43]:
df.shape

(20, 7)

In [44]:
df.content[0]

'為解決急診壅塞，健保署提出增加急診及重症支付標準、給急診護理人員加薪、強化離島急救責任醫院等4招解方，醫院總額研商會今天通過，最快5月上路，年挹注逾新台幣42億元。急診近來面臨前所未有壅塞，衛生福利部中央健康保險署今天在醫療給付費用醫院總額研商議事會討論解方，包含急診支付標準與留觀床支付、急救責任醫院加護病房給付、強化離島急救責任醫院給付與急性一般病床護理費調升等4案。健保署長石崇良今天晚間接受媒體聯訪表示，與解決急診壅塞有關的4項提案全數通過，合計挹注42.35億元，將儘速送入共擬會與健保會，拚5月起開始執行。這次不是只挹注，還有附帶任務指標，共同指標是醫院必須落實醫護人員加薪，其他還有急診壅塞指標改善、加護病房開床率等。針對健保署7日開會與醫界未達成共識的急性一般病床護理費調整，石崇良表示，確定醫學中心第1天住院急性一般病床護理費調幅12.4%，1027點調至1154點，區域醫院調幅14%，905點調至1032點，地區醫院調14.9%，從850點調至977點。石崇良說，醫學中心第2天住院急性一般病床護理費調幅12.3%，從790點調至888點，區域醫院調幅14.1%，696點調至794點，地區醫院調15%，從654點調至752點，挹注24.97億元，盼改善護理薪資環境，留住護理人力，避免關床，以利解決急診患者排隊等住院。這次還有通過強化離島急救責任醫院，石崇良表示，未來將新增離島一般急救責任醫院給付調升標準，所有急診案件皆按照表定點數加給10%，離島中度急救責任醫院所有急診案件給付增加30%、離島重度急救責任醫院調升50%，鼓勵離島醫院提升緊急醫療能力。石崇良說，急診醫師診察費和護理費確定拆分，按照檢傷級數差異，各調增10%，以最嚴重的檢傷分類1級為例，診察費由1223點調到1345點，護理費從577點調到635點；觀察床診察費調至468點，護理費調升6成，留觀第1天自571點調升至914點、第2天從175點調至703點，增預算11億元。急救責任醫院加護病房給付，石崇良說，若是全重度級的急救責任醫院，加護病房給付比照醫學中心；中度級急救責任醫院則比照區域醫院；盼52家重度級急救責任醫院中26家區域醫院，能協助分擔重症病人收治，健保預估新增約5.87億元支出。除了解決急診壅塞，這次研商議事會考量因應台灣少子化與產婦高齡化，高風險生產個案將轉至醫院，通過醫院生產給

In [45]:
df.to_csv("cna_category_news.csv", sep="|", index=False)